### Update pandas a litle bit

In [ ]:
! sudo pip install pandas==0.23

### Download data

In [ ]:
! wget "https://mike0svstorage.blob.core.windows.net/hw1/train" -O /data/train.tar
! wget "https://mike0svstorage.blob.core.windows.net/hw1/test" -O /data/test.tar

In [ ]:
! wget "https://mike0svstorage.blob.core.windows.net/hw1/channels.json" -O /data/channels.json

### Unzip data

In [ ]:
! tar -xvf /data/train.tar -C /data/
! tar -xvf /data/test.tar -C /data/

### Put data into hdfs

In [ ]:
! hdfs dfs -put /data/hw1_train_g/ ./
! hdfs dfs -put /data/hw1_test_x_g/ ./

### Start Spark Application

In [ ]:
import os, sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2.1"
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], 'python'))
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], 'python/lib/py4j-0.10.4-src.zip'))

In [ ]:
import pyspark
import pyspark.sql

In [ ]:
sparkConf = pyspark.SparkConf() \
    .set("spark.executor.memory", "2560m")\
    .set("spark.driver.memory", "2560m")\
    .set("spark.yarn.executor.memoryOverhead", 3584)\
    .set("spark.yarn.driver.memoryOverhead", 3584)\
    .set("spark.python.worker.memory", "1536m")\
    .set("spark.executor.instances", 11)\
    .set("spark.default.parallelism", 300)

In [ ]:
ss = pyspark.sql.SparkSession.builder.config(conf=sparkConf).appName('hw1')\
 .master('yarn').getOrCreate()
ss

### Read data

In [ ]:
train_data = ss.read.json('hw1_train_g').cache()
test_data_x = ss.read.json('hw1_test_x_g').cache()

### Create features

In [ ]:
features_train = ...
features_test = ...

In [ ]:
features_train.write.csv('train_csv', mode='overwrite', header=True)
features_test.write.csv('train_csv', mode='overwrite', header=True)

### Build model

In [ ]:
import subprocess
import glob
import os
import shutil

def load_and_merge_csv_from_hdfs(path, **kwargs):
    shutil.rmtree('/data/{}'.format(os.path.basename(path)))
    get_out = subprocess.check_output('hdfs dfs -get {} /data/'.format(path), shell=True)
    dfs = []
    for g in glob.glob('/data/{}/*.csv'.format(os.path.basename(path))):
        dfs.append(pd.read_csv(g, **kwargs))
    res = pd.concat(dfs)
    res = res.set_index('id')
    return res

In [ ]:
trainXY = load_and_merge_csv_from_hdfs('train_csv')
testX = load_and_merge_csv_from_hdfs('test_csv')

In [ ]:
%pylab inline
import numpy as np

Ycol = 'views'
to_drop = []
trainX, trainY = trainXY.drop(Ycol, axis=1).drop(to_drop, axis=1), trainXY[Ycol]
trainY = np.log(trainY + 100)

testX = testXY.drop(Ycol, axis=1).drop(to_drop, axis=1)

In [ ]:
model = ...

In [ ]:
model.fit(trainX, trainY)

In [ ]:
prediction = model.predict(testX)

In [ ]:
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae

def rmspe(pred, true):
    percent = abs(true - pred).astype(float) / true * 100.
    return (percent ** 2).mean() ** .5

def mape(pred, true):
    percent = abs(true - pred).astype(float) / abs(true) * 100.
    return percent.mean()


In [ ]:
! curl -H 'Cache-Control: no-cache' https://raw.githubusercontent.com/mike0sv/lsml_submit_server/master/src/client.py -o client.py

In [ ]:
import client
reload(client)

In [ ]:
client.make_eval(prediction)